# Lecture 6 - Classification & Regression II (Decision Trees)

Decision Trees are a fundamental model used for classification and regression. While they typically do not yield state-of-the art performances, their inner workings lay the foundation towards more sophisticated models base on Tree Ensembles. Another important advantage of Decision Trees is that they are typically easy to interpret -- that is, by looking at the tree one can see and comprehend what pattern the model was learning.

In general, Decision Trees can handle numerical and categorical. However, scikit-learn implementation "does not support categorical variables for now." (see [documentation](https://scikit-learn.org/stable/modules/tree.html)). Of such detail you have to be aware off when applying off-the-shelf implementations of classification or regression algorithms on your own data. For example, a categorical feature that "looks" like a number such as `postal_code` will be treated as a numerical features when using the `DecisionTreeClassifier` or the `DecisionTreeRegressor` provided by scikit-learn. While the model will train without errors, the result will be off due to the misinterpretation of the data.

As you will see in the examples below, `DecisionTreeClassifier` and `DecisionTreeRegressor` will only create binary decision trees, i.e., each non-leaf node will only have 2 child subtrees. 


Note that Decision Trees do not require the data to be normalized since each decision (i.e., node in the tree) is based on only a single feature. On the other hand, this also means that Decision Trees do not consider the relationship between features. We will explore the consequences in this notebook.


Let's get started...


## Setting up the notebook

Specify how plots get rendered

In [ ]:
%matplotlib notebook

Make all required imports.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split

from matplotlib.colors import ListedColormap

from sklearn.metrics import f1_score, mean_squared_error

#### Some auxiliary Code

In [ ]:
# All classification datasets in this notebook have no more the 3 labels, so 3 colors is enough
colors = ['blue', 'red', 'green']

# Method to plot the decision boundaries (for classification)
# Only applicable if there are 2 input features
def plot_decision_boundaries(clf, X, y, resolution=0.01):

    plt.figure()
    margin = 0.05
    x_min, x_max = X[:, 0].min() - margin, X[:, 0].max() + margin
    y_min, y_max = X[:, 1].min() - margin, X[:, 1].max() + margin
    xx, yy = np.meshgrid(np.arange(x_min, x_max, resolution), np.arange(y_min, y_max, resolution))
    cmap = ListedColormap(colors[:len(np.unique(y))])
    Z = clf.predict(np.array([xx.ravel(), yy.ravel()]).T)
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.4, cmap=cmap)

    plt.scatter(X[:,0], X[:,1], c=[colors[int(c)] for c in y], s=100)
    plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)   
    plt.tight_layout()
    plt.show()

## Working with Toy Data

To better understand the basic characteristics of Decision Trees, we first look at the 2 small examples covered in the lecture.

### Linear Relationships Between Features

The first example is a small classification dataset comprising 26 data points and 2 features. 

#### Create and Visualize Data

In [ ]:
data = np.array([
    (0.05, 0.65, 0), (0.65, 0.2, 0), (0.15, 0.5, 0), (0.25, 0.55, 0), (0.2, 0.4, 0), (0.3, 0.35, 0),
    (0.4, 0.45, 0), (0.45, 0.35, 0), (0.5, 0.25, 0), (0.85, 0.05, 0), (0.6, 0.3, 0), (0.7, 0.25, 0),
    (0.85, 0.3, 1), (0.05, 0.95, 1), (0.2, 0.9, 1), (0.35, 0.85, 1), (0.4, 0.7, 1), (0.5, 0.65, 1), 
    (0.1, 0.85, 1), (0.6, 0.5, 1), (0.7, 0.45, 1), (0.8, 0.4, 1), (0.25, 0.7, 1), (0.35, 0.85, 1), 
    (0.7, 0.6, 1), (0.8, 0.5, 1), 
])

X = data[:,0:2]
y = data[:,2]

num_samples, num_features = X.shape

print('The dataset consists of {} data points, each with {} features.'.format(num_samples, num_features))

Plot the data points.

In [ ]:
plt.figure()
plt.scatter(X[:,0], X[:,1], c=[colors[int(c)] for c in y], s=100)
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)   
plt.tight_layout()
plt.show()

Just from looking at this plot, we can see that the classes could be easily separated by a diagonal line, as there is some linear relationship between the features. However, Decision Tress do not capture such relationships between features as each decision is based only on a single feature.

#### Train a Decision Tree Classifier

Since we have numerical values only, we can use the [Decision Tree implementation of scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). The implementation considers a wide range of input parameters, but we consider here only 2: `max_depth` to specify the maximum depth of the Decision Tree, and `criterion` to specify which scoring function to use to find the best split.

Try changing `max_depth` and see how the resulting Decision Tree looks like. Since this is a very small dataset, it won't be very deep anyway; `max_depth=100` is just to guarantee it's maximum size.

In [ ]:
clf = DecisionTreeClassifier(max_depth=100, criterion='gini').fit(X, y)

print('The Decision Tree has {} nodes.'.format(clf.tree_.node_count))

plt.figure()
tree.plot_tree(clf)
plt.show()

The figure of the Decision Tree above gives you already useful insights. For example, as the second features `X[1]` it's used in the root node, this features is the mos "valuable" since it creates the best first split of complete dataset. The figures also shoes the respective thresholds, e.g., `0.575` in case of the root node.

#### Plot Decision Boundaries

Again, try different values for `max_depth` and see how the decision boundaries change.

In [ ]:
plot_decision_boundaries(clf, X, y)

We can see, that Decision Trees can only generate decision boundaries made out of vertical and horizontal sections -- in the context of this plot. Each section represent a single decision, i.e., a single node in the Decision Tree. That means, that any more intricate decision boundary has to estimated by a series of simple decision boundaries, potentially required large/deep Decision Trees.

### Overfitting & Underfitting

We now perform the same steps as above for a different toy dataset to illustrate the notion of overfitting and underfitting in the context of Decision Trees. This dataset again reflects the example used in the lecture.

#### Create and Visualize Data

In [ ]:
data = np.array([
    (0.05, 0.4, 0), (0.15, 0.1, 0), (0.15, 0.35, 0), (0.2, 0.25, 0), (0.4, 0.4, 0), (0.45, 0.3, 0), 
    (0.95, 0.4, 1), (0.8, 0.4, 1), (0.65, 0.05, 0), (0.7, 0.15, 0), (0.85, 0.1, 0), (0.8, 0.3, 1),
    (0.6, 0.42, 0), (0.4, 0.1, 0), (0.63, 0.32, 0),
    (0.1, 0.55, 1), (0.08, 0.7, 1), (0.32, 0.55, 1), (0.53, 0.75, 1), (0.25, 0.78, 1), (0.9, 0.9, 1),
    (0.38, 0.85, 1), (0.65, 0.9, 1), (0.95, 0.6, 1), (0.80, 0.55, 1), (0.55, 0.6, 1), (0.05, 0.85, 1),
    (0.85, 0.7, 1), (0.32, 0.89, 1), (0.95, 0.05, 0), (0.95, 0.15, 0), (0.92, 0.3, 1)
])

# Add "outlier" point
data = np.concatenate((data, np.array([(0.32, 0.7, 0)])))

X = data[:,0:2]
y = data[:,2]

num_samples, num_features = X.shape

print('The dataset consists of {} data points, each with {} features.'.format(num_samples, num_features))

Plot the data points.

In [ ]:
plt.figure()
plt.scatter(X[:,0], X[:,1], c=[colors[int(c)] for c in y], s=100)
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)   
plt.tight_layout()
plt.show()

#### Train a Decision Tree Classifier

As a above, play with the value of `max_depth` and see how the resulting Decision Tree looks like.

In [ ]:
clf = DecisionTreeClassifier(max_depth=3, criterion='gini')

clf.fit(X, y)

print('The Decision Tree has {} nodes.'.format(clf.tree_.node_count))

plt.figure()
tree.plot_tree(clf)
plt.show()

#### Plot Decision Boundaries

Using different values for `max_depth` will again change the decision boundaries. See how different values effect the area around the outlier.

In [ ]:
plot_decision_boundaries(clf, X, y)

## Decision Tree Classification of IRIS Dataset

The [IRIS dataset](https://archive.ics.uci.edu/ml/datasets/iris) is a very simple classification data for prediction the type of iris plant given 4 numerical features (all lengths in cm).

### Load Data

In [ ]:
df = pd.read_csv('data/iris.csv')

# Convert the species name to numerical categories 0, 1, 2
df['species'] = pd.factorize(df['species'])[0]

# Show the first 5 columns
df.head()

### Case 1: 2 Features

#### Create Training and Test Data

To allow to visualize things more easily, we first consider only two input features (sepal length and sepal width)

In [ ]:
# Convert data to numpy arrays
X = df[['sepal_length', 'sepal_width']].to_numpy()
y = df[['species']].to_numpy().squeeze()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Size of training set: {}".format(len(X_train)))
print("Size of test: {}".format(len(X_test)))
print(len(X_test), len(y_test))

#### Train a Decision Tree Classifier

**Important:** Decision Trees (at least the scikit-learn implementations) have random element, since different features and their respective thresholds might result in the same information gain. That means, two or more splits are equally good, and it depends on the order in which features are processed which feature will "win".

The order of features is randomly permutated in case of scikit-learns `DecisionTreeClassifier` and `DecisionTreeRegressor`. So without setting the `random_state` two runs on the same data might yield different Decision Trees.

In [ ]:
clf = DecisionTreeClassifier(max_depth=5, criterion='gini', random_state=10)

clf.fit(X_train, y_train)

print('The Decision Tree has {} nodes.'.format(clf.tree_.node_count))

plt.figure()
tree.plot_tree(clf)
plt.show()

#### Plot Decision Boundaries

In [ ]:
plot_decision_boundaries(clf, X, y)

When considering only the two features `sepal_length` and `sepal_width` we can see that the red and green labels are quite overlapping, i.e., not really separable. And classification model will have problems with that, but particularly challenging for Decision Trees since can only construct arbitrary decision boundaries based on simple boundary sections.

Change `max_depth` and see how the size and shape of the decision boundaries change.

#### Finding the Best Value for `max_depth`

We already know that the size/depth of a Decision Tree -- here specified by the hyperparameter `max_depth` -- relates to the notion of overfitting and underfitting. The the following we find the best value for `max_depth` by trying a series of choice the record the resulting f1 score for each classifiers.

**Important:** We are a bit sloppy here since we use the test set for this. As you already know, hyperparameter tuning should be done using a validation set.

In [ ]:
max_depth = 20

# Keep track of depth and f1 scores for plotting
ds, f1s = [], []

# Loop over all values for max_depth
for d in range(1, max_depth+1):
    ds.append(d)
    # Train Decision Tree classifier for current value of max_depth
    clf = DecisionTreeClassifier(max_depth=d, criterion='gini', random_state=10).fit(X_train, y_train)
    # Predict class labels for test set
    y_pred = clf.predict(X_test)
    # Calculate f1 score between predictions and ground truth
    f1 = f1_score(y_test, y_pred, average='macro')
    f1s.append(f1)
    
print('A maximum depth of {} yields the best f1 score of {:.3f}'.format(ds[np.argmax(f1s)], np.max(f1s), ))        
    
# Plot the results (max_depth vs. f1.score)
plt.figure()
plt.plot(ds, f1s)
plt.show()

For this example (and with this `random_state`), a `max_depth` of 6 or 7 yields the best results. You can go back and plot the decision boundaries for these values.

### Case 2: 4 Features

Let's now consider all 4 features of the IRIS dataset. Nothing of substance changes, we only can no longer plot the decision boundaries.

#### Create Training and Test Data

In [ ]:
# Convert data to numpy arrays
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].to_numpy()
y = df[['species']].to_numpy().squeeze()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Size of training set: {}".format(len(X_train)))
print("Size of test: {}".format(len(X_test)))
print(len(X_test), len(y_test))

#### Train a Decision Tree Classifier

In [ ]:
clf = DecisionTreeClassifier(max_depth=5, criterion='gini', random_state=10)

clf.fit(X_train, y_train)

print('The Decision Tree has {} nodes.'.format(clf.tree_.node_count))

plt.figure()
tree.plot_tree(clf)
plt.show()

#### Finding the Best Value for `max_depth`

In [ ]:
max_depth = 20

# Keep track of depth and f1 scores for plotting
ds, f1s = [], []

# Loop over all values for max_depth
for d in range(1, max_depth+1):
    ds.append(d)
    # Train Decision Tree classifier for current value of max_depth
    clf = DecisionTreeClassifier(max_depth=d, criterion='gini', random_state=10).fit(X_train, y_train)
    # Predict class labels for test set
    y_pred = clf.predict(X_test)
    # Calculate f1 score between predictions and ground truth
    f1 = f1_score(y_test, y_pred, average='macro')
    f1s.append(f1)
    
print('A maximum depth of {} yields the best f1 score of {:.3f}'.format(ds[np.argmax(f1s)], np.max(f1s), ))
    
plt.figure()
plt.plot(ds, f1s)
plt.show()

Now the results look much better, perfect in fact, although this will slightly differ for different splits in to training an test data. The reason for this is because the IRIS dataset is rather simple and the data points can be separated quite easily when considering all 4 features.

## Decision Tree Regression of Hitters Dataset

Lastly, let's look at an example using Decision Trees for regression.

This [Hitters dataset](https://www.kaggle.com/floser/hitters) was originally taken from the StatLib library which is maintained at Carnegie Mellon University. This is part of the data that was used in the 1988 ASA Graphics Section Poster Session. The salary data were originally from Sports Illustrated, April 20, 1987. The 1986 and career statistics were obtained from The 1987 Baseball Encyclopedia Update published by Collier Books, Macmillan Publishing Company, New York.


### Load Data

The dataset has a couple of categorical features. But for this example, it's not a problem since all of them a binary so mapping them to 0 and 1 is a solution. Note that there is no difference whether 0/1 are treated as a categorical or numerical feature values

In [ ]:
df = pd.read_csv('data/hitters.csv')
df = df.dropna()

df['League'] = pd.factorize(df['League'])[0]
df['NewLeague'] = pd.factorize(df['NewLeague'])[0]
df['Division'] = pd.factorize(df['Division'])[0]

df.head()

### Create Training and Test Data

In [ ]:
X = df[['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat', 'CHits', 'CHmRun', 'CRuns', 'CRBI', 
        'CWalks', 'League', 'Division', 'PutOuts', 'Assists', 'Errors']].to_numpy()

y = df[['Salary']].to_numpy().squeeze()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Size of training set: {}".format(len(X_train)))
print("Size of test: {}".format(len(X_test)))
print(len(X_test), len(y_test))

### Train a Decision Tree Regressor

Training a regressor is basically the same as training a classifier. We have seen in the lecture that A Decision Tree for regression and in for classification are very similar; the core differences is only in the calculation of the impurity since we now have real values instead of labels as outputs.

In [ ]:
reg = DecisionTreeRegressor(max_depth=5, random_state=10).fit(X_train, y_train)

print('The Decision Tree has {} nodes.'.format(reg.tree_.node_count))

plt.figure()
tree.plot_tree(reg)
plt.show()

### Finding the Best Value for `max_depth`

We can use almost the same code as above to find the best value of `max_depth`. We only have to change the evaluation metric from f1 to RSME.

In [ ]:
max_depth = 20

# Keep track of depth and RSMEs for plotting
ds, rsmes = [], []

for d in range(1, max_depth+1):
    ds.append(d)
    # Train Decision Tree regressor for current value of max_depth
    reg = DecisionTreeRegressor(max_depth=d, random_state=10).fit(X_train, y_train)
    # Predict output values for test set
    y_pred = reg.predict(X_test)
    # Calculate RSME between predictions and ground truth
    rsme = mean_squared_error(y_test, y_pred, squared=False)
    rsmes.append(rsme)
    
    
print('A maximum depth of {} yields the best RSME of {:.3f}'.format(ds[np.argmin(rsmes)], np.min(rsmes), ))    
    
plt.figure()
plt.plot(ds, rsmes)
plt.show()

Based on the plot above the  best value for `max_depth` is 12.

## Summary

This notebook introduced and experimented with Decision Trees. The big advantage of Decision Trees is that one can directly inspect and interpret the resulting tree -- this becomes much harder to impossible for tree ensembles. So even you plan to use tree ensembles, as they typically perform better, it is always to good idea to first train single Decision Trees (with different parameter). It gives a first idea about the results but can also help finding good initial values for hyperparameter tuning for the more complex tree ensembles.